# NaOH Absorber Model

Du has collected some very interesting experimental data. In this notebook, we'll do our best to build a 1D model of this system and see how it compares with the available data. 

## Underlying Differential Equations

We will make some standard simplifying assumptions, including a constant gas and liquid volumetric flow rate, adiabatic column behavior, neglible radial concentration or temperature gradients, ideal gas behavior, irreversibility of the reaction of CO2 with OH-, we ignore gas-side mass transfer resistance for CO2 transfer. We will track the concentration of CO2 and H2O in the gas phase, and the *total* concentration of CO2 in the liquid phase, which will be dominated by chemically-bound CO2 in the form of CO32-.  The reaction of CO2 with OH- is given by

$$
\mathrm{CO_2 + 2OH^- \rightarrow CO_3^{2-} + H_2O}
$$

A mass balance for CO2 in the gas phase ( assuming mass transport is dominated by liquid-side resistance) gives:

$$
v_G \mathrm{\left[\frac{m}{s}\right]}\frac{dc_G}{dz} \mathrm{\left[\frac{mol}{m^4}\right]} = -E\mathrm{\left[\cdot\right]}k_L\mathrm{\left[\frac{m}{s}\right]} a\mathrm{\left[\frac{1}{m}\right]}H\mathrm{\left[\frac{mol}{m^3.Pa}\right]}R\mathrm{\left[\frac{Pa. m^3}{K.mol}\right]}T_G\mathrm{\left[K\right]}c_G \mathrm{\left[\frac{mol}{m^3}\right]}
$$

Similarly, a mass balance for CO2 stored in the liquid phase gives:

$$
v_L \mathrm{\left[\frac{m}{s}\right]}\frac{dc_L}{dz} \mathrm{\left[\frac{mol}{m^4}\right]} = -E\mathrm{\left[\cdot\right]}k_L\mathrm{\left[\frac{m}{s}\right]} a\mathrm{\left[\frac{1}{m}\right]}H\mathrm{\left[\frac{mol}{m^3.Pa}\right]}R\mathrm{\left[\frac{Pa. m^3}{K.mol}\right]}T_G\mathrm{\left[K\right]}c_G \mathrm{\left[\frac{mol}{m^3}\right]}
$$

A mass balance for H2O in the gas phase (assuming mass transport is dominated by gas-side resistance) gives:

$$
v_G \mathrm{\left[\frac{m}{s}\right]}\frac{dw_G}{dz} \mathrm{\left[\frac{mol}{m^4}\right]} = -k_G\mathrm{\left[\frac{m}{s}\right]} a\mathrm{\left[\frac{1}{m}\right]}R\mathrm{\left[\frac{Pa. m^3}{K.mol}\right]}T\mathrm{\left[K\right]}\left(w_G \mathrm{\left[\frac{mol}{m^3}\right]} - w_G^* \mathrm{\left[\frac{mol}{m^3}\right]}\right)
$$

An energy balance for the gas phase (assuming all reaction energies partition to the liquid phase) gives:

$$
v_G \mathrm{\left[\frac{m}{s}\right]} c_P^G \mathrm{\left[\frac{J}{mol.K}\right]} \rho_G \mathrm{\left[\frac{mol}{m^3}\right]} \frac{dT_G}{dz} \mathrm{\left[\frac{K}{m}\right]} = -h \mathrm{\left[\frac{J}{K.m^2.s}\right]} a\mathrm{\left[\frac{1}{m}\right]}\left(T_G \mathrm{\left[K\right]} - T_L \mathrm{\left[K\right]}\right)
$$

An energy balance for the liquid phase gives:

$$
v_L \mathrm{\left[\frac{m}{s}\right]} c_P^{LM} \mathrm{\left[\frac{J}{kg.K}\right]} \rho_L^M \mathrm{\left[\frac{kg}{m^3}\right]} \frac{dT_L}{dz} \mathrm{\left[\frac{K}{m}\right]} = -h \mathrm{\left[\frac{J}{K.m^2.s}\right]} a\mathrm{\left[\frac{1}{m}\right]}\left(T_G \mathrm{\left[K\right]} - T_L \mathrm{\left[K\right]}\right) + E\mathrm{\left[\cdot\right]}k_L\mathrm{\left[\frac{m}{s}\right]} a\mathrm{\left[\frac{1}{m}\right]}H\mathrm{\left[\frac{mol}{m^3.Pa}\right]}R\mathrm{\left[\frac{Pa. m^3}{K.mol}\right]}T_G\mathrm{\left[K\right]}c_G \mathrm{\left[\frac{mol}{m^3}\right]}\Delta H\mathrm{\left[\frac{J}{mol}\right]} + k_G\mathrm{\left[\frac{m}{s}\right]} a\mathrm{\left[\frac{1}{m}\right]}R\mathrm{\left[\frac{Pa. m^3}{K.mol}\right]}T_G\mathrm{\left[K\right]}\left(w_G \mathrm{\left[\frac{mol}{m^3}\right]} - w_G^* \mathrm{\left[\frac{mol}{m^3}\right]}\right)\Delta H_w\mathrm{\left[\frac{J}{mol}\right]}
$$

We need a number of constitutive equations to close these ODEs. For $a$, $k_L$, and $k_G$ we use the correlations of Onda et al.

$$
    \frac{a}{a_t} = 1 - \exp \left(-1.45 
    \left(\frac{\sigma_c}{\sigma}\right)^{0.75} \left(\frac{\rho_L^Mv_L}{a_t\mu_L} 
    \right)^{0.1} \left(\frac{v_L^2 a_t}{g} \right)^{-0.05} 
    \left(\frac{\rho_L^Mv_L^2}{\sigma a_t}\right)^{0.2} \right)
$$

$$
k_L\left[\frac{m}{s}\right] \left(\frac{\rho_L^M}{\mu_Lg}\right)^{1/3} \left[\frac{s}{m}\right] = 0.0051 \left(\frac{v_L\rho_L^M}{a\mu_L}\right)^{2/3}\left(\frac{\mu_L}{\rho_L^MD_L}\right)^{-1/2} \left(a_tD_p\right)^{0.4}
$$

$$
Sh_G = k_G \left[\frac{mol}{Pa. m^2.s}\right] \left(\frac{RT_G}{a_tD_g}\right)\left[\frac{Pa. m^2.s}{mol}\right] = 2\left(\frac{v_G\rho_G^M}{a_t\mu_G}\right)^{0.7} \left(\frac{\mu_G}{\rho_G^M D_G}\right)^{1/3}(a_tD_p)^{-2} = 2Re_G^{0.7} Sc_G^{1/3}(a_tD_p)^{-2}
$$

By the Chilton-Colburn analogy between heat and mass transfer, we also have,

$$
Nu_G = h \left[\frac{W}{K.m^2.s}\right] \left(\frac{1}{\kappa_G a_t}\right)\left[\frac{K. m^2.s}{W}\right] = 2\left(\frac{v_G\rho_G^M}{a_t\mu_G}\right)^{0.7} \left(\frac{c_P^M\mu_G}{\kappa_G}\right)^{1/3}(a_tD_p)^{-2} = 2Re_G^{0.7} Pr_G^{1/3}(a_tD_p)^{-2}
$$

As we flow down the column, the composition of the fluid and gas will change. For the gas, we will simply assume that the total concentration is

$$
c_{tot, G} = \frac{p}{RT_G} = \mathrm{\frac{100000 Pa}{8.314 J/K.mol \times T_G[K]}}
$$

so that the N2 concentration in the gas is:

$$
c_{N2, G} = c_{tot, G} - c_{G} - w_G
$$

For the fluid, we will calculate the concentration of CO32- assuming the solvent is initially supplied as pure NaOH and some inlet concentration $c_{OH}^0$. We then have

$$
c_{CO32} = c_L
$$

$$
c_{OH} - c_{OH}^0 - 2c_L
$$

$$
c_{Na} = c_{OH}^0 = c_{OH} + 2c_{CO32}
$$

Note that we're implicitly assuming that the density of the fluid is unchanged, and we're also not explicitly tracking the (very small) change in the flow rate of water in the fluid phase.

For the Henry's constant, we start with Versteeg and Van Swaalj's expression for CO2 solubility in water:

$$
H_{CO2,w}\left[\frac{mol}{m^3.Pa}\right] = 3.54\times10^{-7} \exp \left(\frac{2044}{T\mathrm{[K]}}\right)
$$

We then use the expression of Weisenberger and Schumpe to adjust for changes due to the presence of salts in solution:

$$
\log_{10} \left(\frac{H_{CO2,w}}{H}\right) = \sum_i ((h_i + h_{G0} + h_{T}(T_L - 298.15 K))c_i)
$$

When the ions Na+, OH- and CO32- are present, and when the dissolving gas is CO2, we have

$$
\log_{10} \left(\frac{H_{CO2,w}}{H}\right) =  (h_{OH} + h_{CO20} + h_{T,CO20}(T_L - 298.15 K))c_{OH} +  (h_{Na} + h_{CO20} + h_{T,CO20}(T_L - 298.15 K))c_{Na} +  (h_{CO32} + h_{CO20} + h_{T,CO20}(T_L - 298.15 K))c_{CO32}
$$

For these species we have 
- $h_{Na}   = \mathrm{0.114\times10^{-3} m^3/mol}$
- $h_{OH}   = \mathrm{0.0839\times10^{-3} m^3/mol}$
- $h_{CO32} = \mathrm{0.1423\times10^{-3} m^3/mol}$
- $h_{CO20} = \mathrm{-0.0172\times10^{-3} m^3/mol}$
- $h_{T,CO2} = \mathrm{-0.338\times10^{-6} m^3/mol.K}$.

For the second order reaction rate constant between CO2 and OH-, we can use the Astarita correlation,

$$
k_2 \left[\frac{m^3}{mol.s}\right] = 10^{-3}\exp \left(31.3957 - \frac{6665.9912[K]}{T_L [K]} + 0.0001842 \left[\frac{m^3}{mol}\right] I \left[\frac{mol}{m^3}\right]\right)
$$

For the enhancement factor, we use the expression of Wellek et al.

$$
E = 1 + \left(\frac{1}{(E_i-1)^{-1.35} + (E_1-1)^{-1.35}}\right)^{1/1.35}
$$

where the enhancement factor in the limit of instantaneous reactions is given by

$$
E_i = 1 + \frac{D_{OH}c_{OH}}{2D_{CO2}c_{CO2,int}} =  1 + \frac{D_{OH}c_{OH}}{2D_{CO2}p_{CO2}H}
$$

For the diffusivity of CO2, we use the expression from Liu et al, and we'll just use the total Na concentration. I'd also be tempted to use the Stokes-Einstein equation with a viscosity correlation.

$$
D_{CO2} = D_{CO2,w}(1-0.129\times10^{-3}c_{Na})
$$

For the diffusivity of water in CO2, we'll use Versteeg and Van Swaalj again

$$
D_{CO2,w}\mathrm{\left[\frac{m^2}{s}\right]} = 2.35\times 10^{-6}\exp \left(-\frac{2119}{T[K]}\right)
$$

For the diffusivity of OH, we'll just use a Stokes-Einstien relation ignoring the viscosity dependence,

$$
D_{OH} \mathrm{\left[\frac{m^2}{s}\right]}= 5.27\times10^{-9} \frac{T [K]}{298.15}
$$

For $E_1$ we have

$$
E_1 = \frac{\sqrt{Ha}}{\tanh{\sqrt{Ha}}}
$$

where the Hatta number is given by

$$
Ha = \frac{D_{CO2}k_2c_{OH}}{k_L^2}
$$

For the heat capacity, viscosity and H2O diffusivity in the gas we'll just use values for pure N2.

For the heat capacity, viscosity and mass density of the liquid, we'll use values for water at 40C multiplied by the ratio of a 2M NaOH solution at 25C to the value of water at 25C.

 - $\mu_L = 1.0 \times 10^{-3} Pa.s$
 - $\rho_L^M = 1080 kg/m3$
 - $c_P^{LM} = 4000 J/kg.K$

For the equilibrium concentration of water, we will use the ideal gas law,

$$
w_G^*\left[\frac{mol}{m^3}\right] = \frac{p_{w}^*}{RT_G}
$$
For water vapor pressure, we will assume water is 90 mol\% (reasonable for 2M NaOH), and will apply Raoult's law to Stull's Antoine Equation for water

$$
p_{w}^* [Pa] = 10^5\times 0.9\times 10 ^{4.6543 − (1435.264 / (T[K] - 64.848))}
$$


We will begin by implementing this as a 10 point shooting method, using the same solvers as were used in the DOCCSS project code.

Similarly, we will calculate the inlet water concentration in the gas, $w_G$, based on a relative humidity and Stull's Antoine equation

$$
w_G^0 = \text{Relative Humidity} \times p_w^*(x_w = 1.0)/RT_G^0
$$

For the critical surface tension of the packing, we will use a value of $\sigma_c = 0.04 N/m$, based on typical values for plastics, and $\sigma = 0.055 N/m$ for a NaOH solution. 

## Implementation

Functions for 10 point and 50 point shooting methods for this system are implemented in NaOHModel_10Point.jl and NaOHModel_50Point.jl, respectively. The simplest way to use these functions is to generate plots with the 'soldata' matrix provided by the function. A solution object, 'sol', and an object giving details on the optimization problem, 'minsol', are also provided.

In [ ]:
#Import Julia Packages
using DifferentialEquations
using NLsolve
using Plots
using LaTeXStrings
using Plots.PlotMeasures
#Import NaOH Models
include("NaOHModel_10Point.jl")

In [ ]:
# m and p refer to maximum and minimum wetted areas based on the Onda correlation
at1 = 80.6
at1p = 100
at1m = 34.3
at2 = 30
at2p = 48.2
at2m = 20.2

(minsolT1, solT1, soldataT1) = AbsorberModelNaOH(yG0 = 0.089, L = .6, vG = 0.12372*.52, vL = 0.0013988*.4, cOH0 = 1200, at = at1,TG0 =273.15+22, TL0 = 273.15+15, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
minsolT1.iterations
(minsolT3, solT3, soldataT3) = AbsorberModelNaOH(yG0 = 0.182, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT4, solT4, soldataT4) = AbsorberModelNaOH(yG0 = 0.091, L = .6, vG = 0.12372*.13, vL = 0.0013988*.1, cOH0 = 1200, at = at2,TG0 =273.15+22, TL0 = 273.15+18, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT6, solT6, soldataT6) = AbsorberModelNaOH(yG0 = 0.125, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);

(minsolT1p, solT1p, soldataT1p) = AbsorberModelNaOH(yG0 = 0.089, L = .6, vG = 0.12372*.52, vL = 0.0013988*.4, cOH0 = 1200, at = at1p,TG0 =273.15+22, TL0 = 273.15+15, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT3p, solT3p, soldataT3p) = AbsorberModelNaOH(yG0 = 0.182, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2p,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT4p, solT4p, soldataT4p) = AbsorberModelNaOH(yG0 = 0.091, L = .6, vG = 0.12372*.13, vL = 0.0013988*.1, cOH0 = 1200, at = at2p,TG0 =273.15+22, TL0 = 273.15+18, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT6p, solT6p, soldataT6p) = AbsorberModelNaOH(yG0 = 0.125, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2p,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);

(minsolT1m, solT1m, soldataT1m) = AbsorberModelNaOH(yG0 = 0.089, L = .6, vG = 0.12372*.52, vL = 0.0013988*.4, cOH0 = 1200, at = at1m,TG0 =273.15+22, TL0 = 273.15+15, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT3m, solT3m, soldataT3m) = AbsorberModelNaOH(yG0 = 0.182, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2m,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT4m, solT4m, soldataT4m) = AbsorberModelNaOH(yG0 = 0.091, L = .6, vG = 0.12372*.13, vL = 0.0013988*.1, cOH0 = 1200, at = at2m,TG0 =273.15+22, TL0 = 273.15+18, RelHumid = 0.0, a_method = "Over", ΔH=-110000);
(minsolT6m, solT6m, soldataT6m) = AbsorberModelNaOH(yG0 = 0.125, L = .6, vG = 0.12372*.1, vL = 0.0013988*.1, cOH0 = 1200, at = at2m,TG0 =273.15+22, TL0 = 273.15+16, RelHumid = 0.0, a_method = "Over", ΔH=-110000);


In [ ]:
hscale = 0.6
lit_x_T1 = [3.3,2.2,1.1,0.0]/3.3*hscale; #T1
lit_y_T1 = [15,17,20,23].+273.15;#T1

lit_x_T3 = [3.3,2.2,1.1,0.0]/3.3*hscale; #T3
lit_y_T3 = [16,21.5,26.5,29].+273.15; #T3

lit_x_T4 = [4.4,3.3,2.2,1.1,0.0]/4.4*hscale;#T4
lit_y_T4 = [291.15,292.15,293.65,297.15,299.15];#T4

lit_x_T6 = [4.4,3.3,2.2,1.1,0.0]/4.4*hscale; #T6
lit_y_T6 = [16,18,21,26,27.5].+273.15; #T6

exp_x = ([42.67,39.62,36.58,33.53,30.48,27.43,24.38,21.34,18.29,15.24,12.19,9.144,6.1,3.05,0].+4)/100;

exp_y_T4_naive = [24.07, 24.15,24.25,24.12,24.09,24.10,24.17,24.20,24.18,24.23,24.36,24.17,24.22,24.33].+273.15; #original naive approach
error_T4_naive = [0.32,0.37,0.38,0.34,0.37,0.37,0.39,0.41,0.44,0.45,0.49,0.51,0.56,0.58];
error_tcKn = [1,1,1,1,1,1,1,1,1,1,1,1,1,1].*1.5;

exp_y_T1 = [16.06,16.65,16.5,16.77,16.9,16.82,17.77,18.04,18.18,18.6,18.34,19.32,21.45,21.79,21.94].+273.15; #148 mlpm
error_T1 = [0.19, 0.19, 0.18, 0.18, 0.17, 0.17, 0.15, 0.14,0.13,0.12,0.13,0.12,0.05,0.06,0.011];
exp_y_T3 =[20.23,21.33,21.52,21.57,21.76,21.68,22.71,22.82,23.33,24.12,23.76,25.66,27.22,27.91,28.26].+273.15; #37 mLPM 
error_T3 = [0.28,0.33,0.33,0.32,0.33,0.34,0.37,0.35,0.38,0.47,0.38,0.41,0.33,0.16,0.06];
exp_y_T4 = [18.94,19.44,19.63,19.7,19.8,19.84,20.44,20.57,20.83,21.8,20.92,21.9,24.29,25.85,25.82].+273.15; #37 mlpm
error_T4 = [0.09,0.12,0.12,0.11,0.11,0.11,0.11,0.11,0.11,0.18,0.12,0.12,0.16,0.21,0.20];
exp_y_T6 =[18.10,18.89,19.05,19.07,19.22,19.14,20.24,20.41,20.85,21.93,21.25,23.6,25.71,26.95,27.57].+273.15; #37mLPM
error_T6 = [0.28,0.24,0.20,0.19,0.20,0.20,0.18,0.17,0.15,0.16,0.15,0.19,0.27,0.25,0.28];
error_tcK = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1].*1.5;

total_eT4n = (error_T4_naive.^2+error_tcKn.^2).^0.5;
total_eT1 = (error_T1.^2+error_tcK.^2).^0.5;
total_eT3 = (error_T3.^2+error_tcK.^2).^0.5;
total_eT4 = (error_T4.^2+error_tcK.^2).^0.5;
total_eT6 = (error_T6.^2+error_tcK.^2).^0.5;



In [ ]:
MColor = ["#0072BD", "#D95319", "#EDB120", "#7E2F8E", "#77AC30", "#4DBEEE"]


pT4 = plot(soldataT4[:,6], soldataT4[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(exp_y_T4_naive,exp_x./0.6,xerror = total_eT4n, seriestype=:scatter, label = "Experimental Data, Naive Approach", color = MColor[4],markersize=:6, markerstrokewidth = 1, markerstrokecolor = MColor[4] )
plot!(exp_y_T4,exp_x./0.6,xerror = total_eT4, seriestype=:scatter, label = "Experimental Data, Improved Design", markershape=:circle, markersize=:6, color = MColor[3], markerstrokewidth = 1, markerstrokecolor = MColor[3])
plot!(lit_y_T4,lit_x_T4./0.6, seriestype=:scatter, markershape=:circle, markersize=:6, label = "Tontiwachwuthikul et al. T4", color = MColor[2], markerstrokewidth = 0 )


plot(pT4,
    xlabel = L"T_G (K)", 
    ylabel = "Dimensionless Height", 
    size = (600, 400), 
    dpi = 300,
    framestyle=:box, 
    tickfont = font(12, "serif-roman"), 
    fontfamily = "serif-roman", 
    guidefontsize = 14, 
    grid=false,
    xlimits=(288,304),
    xticks = 288:4:304,
    legend=:bottomleft,
    legendfontsize = 7,
    margin = 5mm,
    )


#savefig("pilotexperiment.svg")


In [ ]:
MColor = ["#0072BD", "#D95319", "#EDB120", "#7E2F8E", "#77AC30", "#4DBEEE"]

pT1 = plot(soldataT1[:,6], soldataT1[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(exp_y_T1,exp_x./0.6,xerror = total_eT1, seriestype=:scatter, label = "Scaled Experiment", markershape=:circle, markersize=:6, color = MColor[3], markerstrokewidth = 1, markerstrokecolor = MColor[3])
plot!(lit_y_T1,lit_x_T1./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [16], T1", color = MColor[2], markerstrokewidth = 0, ylabel = "Dimensionless Height", )

pT3 = plot(soldataT3[:,6], soldataT3[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(exp_y_T3,exp_x./0.6,xerror = total_eT3, seriestype=:scatter, label = "Scaled Experiment", markershape=:circle, markersize=:6, color = MColor[3], markerstrokewidth = 1, markerstrokecolor = MColor[3])
plot!(lit_y_T3,lit_x_T3./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [16], T3", color = MColor[2], markerstrokewidth = 0 )

pT4 = plot(soldataT4[:,6], soldataT4[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(exp_y_T4,exp_x./0.6,xerror = total_eT4, seriestype=:scatter, label = "Scaled Experiment", markershape=:circle, markersize=:6, color = MColor[3], markerstrokewidth = 1, markerstrokecolor = MColor[3])
plot!(lit_y_T4,lit_x_T4./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [16], T4", color = MColor[2], markerstrokewidth = 0 )

pT6 = plot(soldataT6[:,6], soldataT6[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(exp_y_T6,exp_x./0.6,xerror = total_eT6, seriestype=:scatter, label = "Scaled Experiment", markershape=:circle, markersize=:6, color = MColor[3], markerstrokewidth = 1, markerstrokecolor = MColor[3])
plot!(lit_y_T6,lit_x_T6./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [16], T6", color = MColor[2], markerstrokewidth = 0 )

plot(pT1, pT3, pT4, pT6,
    xlabel = L"T_G (K)",  
    size = (1200, 500), 
    dpi = 300,
    framestyle=:box, 
    tickfont = font(12, "serif-roman"), 
    fontfamily = "serif-roman", 
    guidefontsize = 14, 
    grid=false,
    xlimits=(288,304),
    xticks = 288:4:304,
    legend=:topright,
    legendfontsize = 7,
    margin = 5mm,
    title = ["(a)" "(b)" "(c)" "(d)"], titleloc = :left,
    layout = (1,4)
    )


#savefig("T1346_plot.svg")


In [ ]:
MColor = ["#0072BD", "#D95319", "#EDB120", "#7E2F8E", "#77AC30", "#4DBEEE"]

pT1 = plot(soldataT1[:,6], soldataT1[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT1p[:,6], soldataT1p[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT1m[:,6], soldataT1m[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(lit_y_T1,lit_x_T1./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [X], T1", color = MColor[2], markerstrokewidth = 0, ylabel = "Dimensionless Height", )

pT3 = plot(soldataT3[:,6], soldataT3[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT3p[:,6], soldataT3p[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT3m[:,6], soldataT3m[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(lit_y_T3,lit_x_T3./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [X], T3", color = MColor[2], markerstrokewidth = 0 )

pT4 = plot(soldataT4[:,6], soldataT4[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT4p[:,6], soldataT4p[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT4m[:,6], soldataT4m[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(lit_y_T4,lit_x_T4./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [X], T4", color = MColor[2], markerstrokewidth = 0 )

pT6 = plot(soldataT6[:,6], soldataT6[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT6p[:,6], soldataT6p[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(soldataT6m[:,6], soldataT6m[:,1]./0.6, label = "Scaled Model", color = MColor[1], linewidth =3)
plot!(lit_y_T6,lit_x_T6./0.6, seriestype=:scatter, markershape=:square, markersize=:6, label = "Pilot Ref. [X], T6", color = MColor[2], markerstrokewidth = 0 )


plot(pT1, pT3, pT4, pT6,
    xlabel = L"T_G (K)",  
    size = (1200, 500), 
    dpi = 300,
    framestyle=:box, 
    tickfont = font(12, "serif-roman"), 
    fontfamily = "serif-roman", 
    guidefontsize = 14, 
    grid=false,
    xlimits=(288,306),
    xticks = 288:4:304,
    legend=:topright,
    legendfontsize = 7,
    margin = 5mm,
    title = ["(a)" "(b)" "(c)" "(d)"], titleloc = :left,
    layout = (1,4)
    )


#savefig("T1346_plot_CA.svg")